In [266]:
import pandas as pd

df = pd.read_csv("2020_us_state_finances.csv")

df['Description'] = df['Description'].str.strip()

# Retains only columns that combines state and local government finances
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Retains only required rows
df = df[df["Description"].isin(["Expenditure1", "Assistance and subsidies", "Public welfare"])]

# Transposes dataframe and rename columns
df = df.transpose()
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df.reset_index(inplace=True)
df.rename(columns = {'Expenditure1':'State and Local Government Total Expenditure', "index" : "State", "Assistance and subsidies":"Expenditure on Assistance and Subsidies", "Public welfare":"Expenditure on Public Welfare"}, inplace = True)

# Removes first row with total US state and local government expenditure
df.drop(index=df.index[0], axis=0, inplace=True)

df.to_csv("cleaned_data", index = False)

In [267]:
pd.read_csv("cleaned_data")

,State,State and Local Government Total Expenditure,Expenditure on Assistance and Subsidies,Expenditure on Public Welfare
0,Alabama,"52,646,924","1,076,698","7,806,178"
1,Alaska,"15,076,507","175,657","2,731,308"
2,Arizona,"71,488,388","3,297,468","17,299,467"
3,Arkansas,"28,980,661","529,113","7,365,628"
4,California,"678,462,621","14,641,373","136,898,802"
5,Colorado,"74,023,492","1,135,544","10,780,858"
6,Connecticut,"45,285,851","541,747","3,934,986"
7,Delaware,"13,311,734","401,682","2,815,592"
8,District of Columbia,"23,499,816","887,938","4,393,854"
9,Florida,"208,234,166","3,129,052","30,643,568"
